In [1]:
import sys
import numpy as np
import pandas as pd
import json

sys.path.append('../..')

from heat_load_calc.core import core

# core

## 1. Outline

モジュール core は建物仕様・外気温湿度...(後で詳細に記述）...を与えることで、ステップごとのループ計算を行い、室温度・湿度、暖冷房負荷を計算するモジュールです。

## 2. Interface

calc(input_data_dir, output_data_dir = None, show_simple_result = False, show_detail_result = False)

params:

| パラメータ名　| 型 | 概要 |
| :--- | :--- | :--- |
| input_data_dir | str | 入力データが格納されたディレクトリを指定します。 |
| output_data_dir | str | （省略可能）計算された室温度・湿度、暖冷房負荷等をCSVで出力するフォルダを指定します。CSVを出力しない場合は省略可能。 |
| show_simple_result | bool | （省略可能）計算結果（概要版）をはきだす場合はTrueにします。はきださない場合はFalseを指定しないか省略する。 |
| show_detail_result | bool | （省略可能）計算結果（詳細版）をはきだす場合はTrueにします。はきださない場合はFalseを指定しないか省略する。 |

returns:

計算結果の概要版と詳細版をpandas.DataFrame 形式で返します。  
2つの pandas.DataFrame 形式ファイルを、タプル形式で返します。  
（簡易版計算結果, 詳細版計算結果）

## 3. Input Files

引数 'input_data_dir' で指定したフォルダに以下のファイルを用意しないといけない。

### 1) mid_data_house.json

#### 仕様

json ファイルの仕様

In [2]:
pd.read_csv('../data_spec/spec_mid_data.csv')

,階層1,階層2,階層3,階層4,階層5,階層6,項目名,key,入力必須条件 ○：必須項目 △：上位項目が必須の場合において必須項目,ID:value,ID:意味,型,単位,備考
0,〇,NaN,NaN,NaN,NaN,NaN,空間,spaces,〇,－,－,－,－,NaN
1,NaN,〇,NaN,NaN,NaN,NaN,室通し番号,id,〇,－,－,int,－,NaN
2,NaN,〇,NaN,NaN,NaN,NaN,室名,name,〇,－,－,str,－,NaN
3,NaN,〇,NaN,NaN,NaN,NaN,室気積,volume,〇,－,－,float,m3,NaN
4,NaN,〇,NaN,NaN,NaN,NaN,相当隙間面積,c_value,〇,－,－,float,cm2/m2,NaN
5,NaN,〇,NaN,NaN,NaN,NaN,放射暖房の対流放熱比率,beta,〇,－,－,float,－,NaN
6,NaN,〇,NaN,NaN,NaN,NaN,換気,ventilation,〇,－,－,－,－,NaN
7,NaN,NaN,〇,NaN,NaN,NaN,機械換気流入量,mechanical,〇,－,－,float,m3/sec,NaN
8,NaN,NaN,〇,NaN,NaN,NaN,隣室間換気流入量,next_spaces,〇,－,－,List[float],m3/sec,配列番号と室id番号が対応している
9,NaN,NaN,〇,NaN,NaN,NaN,自然風利用時の換気量,natural,〇,－,－,float,m3/sec,NaN


#### json ファイルの基本構成

基本的なjsonファイルの書き方は以下のとおり。

{  
    'spaces': [  
        {'id': 0, ...その他の情報...},  
        {'id': 1, ...その他の情報...},  
        ...  
        {'id': 10, ...その他の情報...}  
    ],       
    'boundaries': [
        {'id': 0, ...その他の情報...},  
        {'id': 1, ...その他の情報...},  
        ...  
        {'id': 100, ...その他の情報...}  
    ]  
} 

#### スペース1つ分の書き方の例

In [3]:
# スペース1つ分の書き方例
with open('core_data_tutorial/mid_data_house.json') as f:
    dj = json.load(f)

dj['spaces'][0]

{'id': 0,
 'name': 'main_occupant_room',
 'volume': 70.92,
 'c_value': 2.0,
 'beta': 0.0,
 'ventilation': {'mechanical': 0.016666666666666666,
  'next_spaces': [0.0, 0.0, 0.0],
  'natural': 0.0985},
 'furniture': {'heat_capacity': 893592.0,
  'heat_cond': 196.59024,
  'moisture_capacity': 1191.4560000000001,
  'moisture_cond': 2.1446208},
 'equipment': {'heating': {'radiative': {'installed': False},
   'convective': {}},
  'cooling': {'radiative': {'installed': False},
   'convective': {'q_min': 500.0,
    'q_max': 6049.891511,
    'v_min': 11.086608037753514,
    'v_max': 20.15746915955184}}}}

#### 境界1つ分の書き方の例

In [4]:
with open('core_data_tutorial/mid_data_house.json') as f:
    dj = json.load(f)

dj['boundaries'][0]

{'id': 0,
 'name': 'integrated_boundary0',
 'sub_name': 'north_exterior_wall',
 'is_ground': 'false',
 'connected_space_id': 0,
 'area': 4.9775,
 'phi_a0': 0.07082379236222458,
 'phi_a1': [4.507057985882911e-05,
  -0.0009266420480683226,
  0.015710747070284452,
  0.1371664403861094,
  -0.11163144218064561,
  0.09174099947400259,
  -0.022630779908299824,
  0.0033885297113725936,
  0.0,
  0.0,
  0.0,
  0.0],
 'phi_t0': 0.001598173949854509,
 'phi_t1': [4.014649140609802e-05,
  -0.0008192243722720102,
  0.013605804259968005,
  0.1088867059986907,
  -0.19238689071678203,
  0.10296731413100914,
  -0.022435410952413436,
  0.0029186857277683836,
  0.0,
  0.0,
  0.0,
  0.0],
 'r': [0.9982016190284373,
  0.99291521625209,
  0.9723062034557024,
  0.8950058718522789,
  0.6452076990440302,
  0.17714441059303657,
  0.0010735761574841769,
  1.869406715200079e-12,
  0.0,
  0.0,
  0.0,
  0.0],
 'h_i': 9.090909090909092,
 'flr': 0.0,
 'is_solar_absorbed': 'False',
 'f_mrt_hum': 0.027134347293875108,
 '

### 2) weather.csv

#### 仕様

In [5]:
pd.read_csv('../data_spec/d_weather.csv')

,列名,単位,概要
0,temperature,degree C,外気温度
1,absolute humidity,kg/kg(DA),外気絶対湿度
2,normal direct solar radiation,W/m2,法線面直達日射量
3,horizontal sky solar radiation,W/m2,水平面天空日射量
4,outward radiation,W/m2,夜間放射量
5,sun altitude,rad,太陽高度
6,sun azimuth,rad,太陽方位角


#### 記載例

In [6]:
pd.read_csv('core_data_tutorial/weather.csv').head()

,Unnamed: 0,temperature,absolute humidity,normal direct solar radiation,horizontal sky solar radiation,outward radiation,sun altitude,sun azimuth
0,1989-01-01 00:00:00,2.300,0.003200,0.0,0.0,83.333333,-1.365523,2.988844
1,1989-01-01 00:15:00,2.375,0.003225,0.0,0.0,82.638889,-1.365761,-2.997617
2,1989-01-01 00:30:00,2.450,0.003250,0.0,0.0,81.944444,-1.350635,-2.720263
3,1989-01-01 00:45:00,2.525,0.003275,0.0,0.0,81.250000,-1.322955,-2.488647
4,1989-01-01 01:00:00,2.600,0.003300,0.0,0.0,80.555556,-1.286386,-2.305989


### 3) mid_data_ac_demand.csv

#### 値

True：空調ON  
False：空調OFF

#### 例

In [7]:
pd.read_csv('core_data_tutorial/mid_data_ac_demand.csv', header=None).head()

,0,1,2
0,False,True,False
1,False,True,False
2,False,True,False
3,False,True,False
4,False,True,False


### 4) mid_data_heat_generation.csv

#### 値

機器内部発熱値, W

#### 例

In [8]:
pd.read_csv('core_data_tutorial/mid_data_heat_generation.csv', header=None).head()

,0,1,2
0,135.0,0.867,44.7
1,135.0,0.867,44.7
2,135.0,0.867,44.7
3,135.0,0.867,44.7
4,10.3,0.867,44.7


### 5) mid_data_local_vent.csv

#### 値

局所換気量, m<sup>3</sup>/s

#### 例

In [9]:
pd.read_csv('core_data_tutorial/mid_data_local_vent.csv', header=None).head()

,0,1,2
0,0.0,0.0,0.0
1,0.0,0.0,0.0
2,0.0,0.0,0.0
3,0.0,0.0,0.0
4,0.0,0.0,0.0


### 6) mid_data_moisture_generation.csv

#### 値

調理内部発湿, kg/s

#### 例

In [10]:
pd.read_csv('core_data_tutorial/mid_data_moisture_generation.csv', header=None).head()

,0,1,2
0,0.0,0.0,0.0
1,0.0,0.0,0.0
2,0.0,0.0,0.0
3,0.0,0.0,0.0
4,0.0,0.0,0.0


### 7) mid_data_occupants.csv

#### 値

在室人員, 人

#### 例

In [11]:
pd.read_csv('core_data_tutorial/mid_data_occupants.csv', header=None).head()

,0,1,2
0,0.0,4.0,0.0
1,0.0,4.0,0.0
2,0.0,4.0,0.0
3,0.0,4.0,0.0
4,0.0,4.0,0.0


### 8) mid_data_q_trs_sol.csv

#### 値

透過日射熱取得[W]

#### 例

In [12]:
pd.read_csv('core_data_tutorial/mid_data_q_trs_sol.csv', header=None).head()

,0,1,2
0,0.0,0.0,0.0
1,0.0,0.0,0.0
2,0.0,0.0,0.0
3,0.0,0.0,0.0
4,0.0,0.0,0.0


### 9) mid_data_theta_o_sol.csv

#### 値

外皮：裏面等価温度, ℃  
内壁の場合は裏面等価温度は0.0℃としている。

#### 例

In [13]:
pd.read_csv('core_data_tutorial/mid_data_theta_o_sol.csv', header=None).head()

,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
0,0.8000,0.8000,0.8000,0.8000,0.8000,0.8000,2.300,-4.45000,0.0,0.0,...,0.0,0.8000,0.8000,0.8000,0.8000,0.8000,2.300,-4.45000,0.0,0.0
1,0.8875,0.8875,0.8875,0.8875,0.8875,0.8875,2.375,-4.31875,0.0,0.0,...,0.0,0.8875,0.8875,0.8875,0.8875,0.8875,2.375,-4.31875,0.0,0.0
2,0.9750,0.9750,0.9750,0.9750,0.9750,0.9750,2.450,-4.18750,0.0,0.0,...,0.0,0.9750,0.9750,0.9750,0.9750,0.9750,2.450,-4.18750,0.0,0.0
3,1.0625,1.0625,1.0625,1.0625,1.0625,1.0625,2.525,-4.05625,0.0,0.0,...,0.0,1.0625,1.0625,1.0625,1.0625,1.0625,2.525,-4.05625,0.0,0.0
4,1.1500,1.1500,1.1500,1.1500,1.1500,1.1500,2.600,-3.92500,0.0,0.0,...,0.0,1.1500,1.1500,1.1500,1.1500,1.1500,2.600,-3.92500,0.0,0.0


## Output

### 簡易版計算結果

In [14]:
pd.read_csv('../data_spec/spec_core_output_simple.csv')

,出力結果の種類,"列名 ([i]=室ID, [j]=境界ID）",単位,概要
0,室,rm[i]_ac_operate,－,運転状態
1,室,rm[i]_t_r,℃,空気温度
2,室,rm[i]_x_r,kg/kg(DA),室絶対湿度
3,室,rm[i]_ot,℃,室作用温度
4,室,rm[i]_l_s_c,W,対流空調顕熱負荷
5,室,rm[i]_l_s_r,W,放射空調顕熱負荷
6,室,rm[i]_l_l_c,W,対流空調潜熱負荷


### 詳細版計算結果

In [15]:
pd.read_csv('../data_spec/spec_core_output_detail.csv')

,出力結果の種類,"列名 ([i]=室ID, [j]=境界ID）",単位,概要
0,全体,out_temp,℃,外気温度
1,全体,out_abs_humid,kg/kg(DA),外気絶対湿度
2,室,rm[i]_ac_operate,－,運転状態
3,室,rm[i]_occupancy,boolean,在室状況
4,室,rm[i]_pmv_target,－,目標PMV
5,室,rm[i]_hc_hum,W/(m2･K),在室者周りの対流熱伝達率
6,室,rm[i]_hr_hum,W/(m2･K),在室者周りの放射熱伝達率
7,室,rm[i]_v_hum,m/s,在室者周りの風速
8,室,rm[i]_t_r,℃,空気温度
9,室,rm[i]_rh_r,%,室相対湿度


## Example

In [16]:
ds, dd = core.calc(input_data_dir='core_data_tutorial')

助走計算（土壌のみ）
助走計算（建物全体）
本計算
ログ作成


In [17]:
ds.head()

,out_temp,out_abs_humid,rm0_ac_operate,rm0_t_r,rm0_x_r,rm0_ot,rm0_l_s_c,rm0_l_s_r,rm0_l_l_c,rm1_ac_operate,...,rm1_l_s_c,rm1_l_s_r,rm1_l_l_c,rm2_ac_operate,rm2_t_r,rm2_x_r,rm2_ot,rm2_l_s_c,rm2_l_s_r,rm2_l_l_c
1989-01-01 00:00:00,2.41,0.0,OperationMode.STOP_CLOSE,14.97,0.0040,13.97,0.0,0.0,0.0,OperationMode.HEATING,...,11067.0,0.0,0.0,OperationMode.STOP_CLOSE,5.62,0.1644,6.27,0.0,0.0,0.0
1989-01-01 01:00:00,2.71,0.0,OperationMode.STOP_CLOSE,12.75,0.0040,11.99,0.0,0.0,0.0,OperationMode.HEATING,...,11632.0,0.0,0.0,OperationMode.STOP_CLOSE,5.27,0.1673,5.89,0.0,0.0,0.0
1989-01-01 02:00:00,2.86,0.0,OperationMode.STOP_CLOSE,11.25,0.0040,10.66,0.0,0.0,0.0,OperationMode.HEATING,...,11978.0,0.0,0.0,OperationMode.STOP_CLOSE,4.95,0.1702,5.56,0.0,0.0,0.0
1989-01-01 03:00:00,2.65,0.0,OperationMode.STOP_CLOSE,10.12,0.0039,9.67,0.0,0.0,0.0,OperationMode.HEATING,...,12337.0,0.0,0.0,OperationMode.STOP_CLOSE,4.68,0.1730,5.27,0.0,0.0,0.0
1989-01-01 04:00:00,2.40,0.0,OperationMode.STOP_CLOSE,9.21,0.0039,8.86,0.0,0.0,0.0,OperationMode.HEATING,...,12708.0,0.0,0.0,OperationMode.STOP_CLOSE,4.41,0.1759,5.00,0.0,0.0,0.0


In [18]:
dd.head()

,out_temp,out_abs_humid,rm0_ac_operate,rm0_occupancy,rm0_pmv_target,rm0_hc_hum,rm0_hr_hum,rm0_t_r,rm0_rh_r,rm0_x_r,...,rm2_b8_qisol_s,rm2_b0_qiall_s,rm2_b1_qiall_s,rm2_b2_qiall_s,rm2_b3_qiall_s,rm2_b4_qiall_s,rm2_b5_qiall_s,rm2_b6_qiall_s,rm2_b7_qiall_s,rm2_b8_qiall_s
1989-01-01 00:00:00,2.300,0.003200,OperationMode.STOP_CLOSE,False,0.0,4.042519,3.971754,15.880388,36.451059,0.004065,...,0.0,2.346373,27.441792,2.164487,2.150024,27.427286,1.052751,1.987786,-22.600060,-21.406218
1989-01-01 00:15:00,2.375,0.003225,OperationMode.STOP_CLOSE,False,0.0,4.086926,3.951726,15.239571,37.874433,0.004053,...,0.0,2.305389,26.525425,2.133448,2.118949,26.511150,0.950580,1.964214,-21.106089,-21.627755
1989-01-01 00:30:00,2.450,0.003250,OperationMode.STOP_CLOSE,False,0.0,4.125052,3.933769,14.655859,39.221203,0.004043,...,0.0,2.298917,25.617437,2.134306,2.120109,25.603377,0.877437,1.936261,-19.812076,-21.841623
1989-01-01 00:45:00,2.525,0.003275,OperationMode.STOP_CLOSE,False,0.0,4.158925,3.917489,14.121641,40.497316,0.004032,...,0.0,2.302100,24.722222,2.140630,2.127223,24.708352,0.827327,1.912196,-18.655625,-22.048890
1989-01-01 01:00:00,2.600,0.003300,OperationMode.STOP_CLOSE,False,0.0,4.189370,3.902798,13.422160,42.278733,0.004022,...,0.0,2.308678,23.857944,2.152452,2.139587,23.844281,0.800124,1.910604,-17.575055,-22.230768
